#### Esse código tem o foco de pensar em como construir rotinas dentro do Glue

In [ ]:
# Configurações
import sys
import pandas as pd
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
import awswrangler as wr
from datetime import datetime, date, timedelta
import boto3

# Inicializar o SparkContext e o GlueContext
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

# Parâmetros do Job
args = getResolvedOptions(sys.argv, ['JOB_NAME'])
job.init(args['JOB_NAME'], args)

In [ ]:
#  Import da base
from pyspark.sql.functions import col

# Pega a base completa antes de semana_passada
select = f"""
SELECT 
    *
FROM NOME_TABELA
"""
# ESSA FUNÇÃO NÃO PRECISA SER ALTERADA, serve para puxar dados do catalogo de dados do athena
df_base = wr.athena.read_sql_query(
    select
    ,database='workspace_db'
    ,ctas_approach=False
    ,workgroup='workgroup'
)
df_spark = spark.createDataframe(df_base)
df_spark.show()

In [ ]:
#  1. Transformação de dados
df = df_spark.withColumn('nome_coluna', col('nome_coluna').cast('int') )

def calculaAprovacao(rating):
    if (rating == 'A1'):
        return 1
    else:
        return 0
# Coloca a função que usaremos e sua saida 
udf_preparado = udf(calculaAprovacao, IntegerType())
# Aplica a transformação de dados que queremos, alicando o resultado na coluna fl
df = df.withColumn('fl', udf_preparado( col('RATING') ) )